<a href="https://colab.research.google.com/github/DajeanArcila/DajeanArcila/blob/main/PROJETO_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


from statsmodels.tsa.arima.model import ARIMA

In [8]:
df=pd.read_csv('Pasta1.csv',sep=';')

In [9]:
df.head()

,Column1,Column2
0,date,value
1,1991-07-01,3.526591
2,1991-08-01,3.180891
3,1991-09-01,3.252221
4,1991-10-01,3.611003


In [10]:
!pip install scipy

In [11]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.5 MB/s eta 0:00:00


In [14]:
from pmdarima.arima import auto_arima

In [15]:
from xgboost import XGBRegressor

In [19]:
df.shape

(205, 2)

In [20]:
df.describe()

,Column1,Column2
count,205,205
unique,205,205
top,date,value
freq,1,1


In [21]:
df.index.min(), df.index.max()

(0, 204)

In [22]:
#df.columns= {'produccion'}

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements